In [ ]:
# Ensure the svt library will auto reload (only really needed if developing svt.py)
%load_ext autoreload

In [ ]:
# Imports
import json
import math
import os
import sys

import numpy as np
import svt
%autoreload

# Seed random number generator for consistency
np.random.seed(0)

ASSET_DIR = os.path.join("..", "..", "ci", "assets")

def asset_path(filename):
    return os.path.join(ASSET_DIR, filename)


In [ ]:
# Tutorial 1 - Scene and Canvas basics

# Create a Scene, the top level container in ScenePic
scene = svt.Scene()

# A Scene can contain many Canvases
# For correct operation, you should create these using scene1.create_canvas() (rather than constructing directly using svt.Canvas(...)) 
canvas_1 = scene.create_canvas_3d(width = 300, height = 300)
canvas_2 = scene.create_canvas_3d(width = 100, height = 300)

# You can print out the underlying ScenePic script for the scene if you like, though only mainly for interest
#print scene.get_script()

# ScenePic has told Jupyter how to display scene objects
#scene.save_as_html("Test.html", embed_library=False)
scene

In [ ]:
# Tutorial 2 - Meshes and Frames

# Create a scene
scene = svt.Scene()

# A Mesh is a vertex/triangle/line buffer with convenience methods
# Meshes "belong to" the Scene, so should be created using create_mesh()
# Meshes can be re-used across multiple frames/canvases
my_first_mesh = scene.create_mesh(shared_color = svt.Color(1.0, 0.0, 1.0)) # If shared_color is not provided, you can use per-vertex coloring
my_first_mesh.add_cube(transform = svt.Transforms.Scale(0.1)) # Adds a unit cube centered at the origin
my_first_mesh.add_cube(transform = np.dot(svt.Transforms.Translate([-1.0, 1.0, -1.0]), svt.Transforms.Scale(0.5)))
my_first_mesh.add_sphere(transform = svt.Transforms.Translate([1.0, 1.0, 1.0]))
#print my_first_mesh.vertex_buffer

# A Canvas is a 3D rendering panel
canvas = scene.create_canvas_3d(width = 300, height = 300)

# Create an animation with multiple Frames
# A Frame references a set of Meshes
# Frames are created from the Canvas not the Scene
for i in range(10):
    frame = canvas.create_frame()
    frame.add_mesh(my_first_mesh, transform = svt.Transforms.Translate([i / 10.0, 0.0, 0.0])) # An arbitrary rigid transform can optionally be specified.
    mesh2 = scene.create_mesh(shared_color = svt.Color(1.0,0.0,0.0),camera_space=True)
    mesh2.add_cube(transform = np.dot(svt.Transforms.Translate([0.0, 0.0, -5.0]), svt.Transforms.Scale(0.5)))
    frame.add_mesh(mesh2)
    label = scene.create_label(text = "Hi", color = svt.Colors.White, size_in_pixels = 80, offset_distance = 0.6, camera_space = True)
    frame.add_label(label = label, position = [0.0, 0.0, -5.0])
    
# Display the Scene in Jupyter
#scene.save_as_html("Debug.html", embed_library=False)
scene

In [ ]:
# Tutorial 3 - Point clouds 1

# Create a scene
scene = svt.Scene()

# Create a mesh that we'll turn in to a point-cloud using enable_instancing()
mesh = scene.create_mesh(shared_color = svt.Color(0,1,0))
mesh.add_cube() # Unit diameter cube that will act as primitive
mesh.apply_transform(svt.Transforms.Scale(0.01)) # Scale the primitive
mesh.enable_instancing(positions = 2 * np.random.rand(10000, 3) - 1) # Cause the mesh to be replicated across many instances with the provided translations.  You can optionally also provide per-instance colors and quaternion rotations.

# Create Canvas and Frame, and add Mesh to Frame
canvas = scene.create_canvas_3d(width = 300, height = 300, shading=svt.Shading(bg_color=svt.Colors.White))
frame = canvas.create_frame()
frame.add_mesh(mesh)

#scene.save_as_html("Temp2.html", embed_library=False)
scene

In [ ]:
# Tutorial 4 - Points clouds 2
# Note that the point cloud primitive can be arbitrarily complex.
# The primitive geometry will only be stored once for efficiency.

# Some parameters
disc_thickness = 0.2
normal_length = 1.5
point_size = 0.1

# A helper Mesh which we won't actually use for rendering - just to find the points and normals on a sphere to be used in mesh2 below
# NB this is created using the svt.Mesh() constructor directly so it doesn't get added automatically to the Scene
sphere_mesh = svt.Mesh()
sphere_mesh.add_sphere(transform = svt.Transforms.Scale(2.0), color = svt.Color(1.0, 0.0, 0.0))
N = sphere_mesh.count_vertices()
points = sphere_mesh.vertex_buffer['pos']
normals = sphere_mesh.vertex_buffer['norm']

# Convert the normals into quaternion rotations
rotations = np.zeros((N, 4))
for i in range(0, N):
    rotations[i, :] = svt.Transforms.QuaternionToRotateXAxisToAlignWithAxis(normals[i, :])

# Generate some random colors
colors = np.random.rand(N,3)
    
# Create a scene
scene = svt.Scene()

# Create a mesh that we'll turn in to a point-cloud using enable_instancing()
mesh = scene.create_mesh(shared_color = svt.Color(0,1,0), double_sided = True) # shared_color will be overridden in a moment

# Add the primitive to the Mesh - a disc and a thickline showing the normal
#mesh.add_cylinder(segment_count = 20, transform = svt.Transforms.Scale([disc_thickness, 1.0, 1.0]))
mesh.add_disc(segment_count = 20, transform = svt.Transforms.Scale([disc_thickness, 1.0, 1.0]))
mesh.add_thickline(start_point = np.array([disc_thickness * 0.5, 0.0, 0.0]), end_point = np.array([normal_length, 0.0, 0.0]), start_thickness = 0.2, end_thickness = 0.1)
mesh.apply_transform(svt.Transforms.Scale(point_size))

# Now turn the mesh into a point-cloud
mesh.enable_instancing(positions = points, rotations = rotations, colors = colors) # Both rotations and colors are optional

# Create Canvas and Frame, and add Mesh to Frame
canvas = scene.create_canvas_3d(width = 300, height = 300)
frame = canvas.create_frame()
frame.add_mesh(mesh)

scene

In [ ]:
# Tutorial 5 - Misc Meshes

# Scene is the top level container in ScenePic
scene = svt.Scene()

# Ok - let's start by creating some Mesh objects

# Mesh 1 - contains a cube and a sphere
# Mesh objects can contain arbitrary triangle mesh and line geometry
# Meshes can belong to "layers" which can be controlled by the user interactively
mesh1 = scene.create_mesh(layer_id = "Sphere+") # No shared_color provided, so per-vertex coloring enabled
mesh1.add_cylinder(color = svt.Color(1.0, 0.0, 0.0), transform = svt.Transforms.Translate([-2.0, 0.0, -2.0]))
mesh1.add_uv_sphere(color = svt.Color(0.0, 0.0, 1.0), transform = np.dot(svt.Transforms.Translate([-1.0, 1.0, 0.0]), svt.Transforms.Scale(1.8)), fill_triangles = False, add_wireframe = True)
mesh1.add_icosphere(color = svt.Color(0.0, 1.0, 1.0), transform = np.dot(svt.Transforms.Translate([2.0, 1.0, 0.0]), svt.Transforms.Scale(1.8)), fill_triangles = False, add_wireframe = True, steps = 2)

# Mesh 2 - coordinate axes
mesh2 = scene.create_mesh(layer_id = "Coords")
mesh2.add_coordinate_axes(transform = svt.Transforms.Translate([0.0, 0.0, 0.0]))

# Mesh 3 - example of Loop Subdivision on a cube
cube_verts = np.array([[-0.5, -0.5, -0.5], [+0.5, -0.5, -0.5], [-0.5, +0.5, -0.5], [+0.5, +0.5, -0.5], [-0.5, -0.5, +0.5], [+0.5, -0.5, +0.5], [-0.5, +0.5, +0.5], [+0.5, +0.5, +0.5]])
cube_tris = np.array([[0, 2, 3], [0, 3, 1], [1, 3, 7], [1, 7, 5], [4, 5, 7], [4, 7, 6], [4, 6, 2], [4, 2, 0], [2, 6, 7], [2, 7, 3], [4, 0, 1], [4, 1, 5]])
cube_verts_a, cube_tris_a = svt.LoopSubdivStencil(cube_tris, 2, False).apply(cube_verts) # Two steps of subdivision, no projection to limit surface.  Stencils could be reused for efficiency for other meshes with same triangle topology.
cube_verts_b, cube_tris_b = svt.LoopSubdivStencil(cube_tris, 2, True).apply(cube_verts) # Two steps of subdivision, projection to limit surface.  Stencils could be reused for efficiency for other meshes with same triangle topology.
mesh3 = scene.create_mesh(shared_color = svt.Color(1.0, 0.8, 0.8))
mesh3.add_mesh_without_normals(cube_verts, cube_tris, transform = svt.Transforms.Translate([-1.0, 0.0, 0.0])) # Add non-subdivided cube
mesh3.add_mesh_without_normals(cube_verts_a, cube_tris_a)
mesh3.add_mesh_without_normals(cube_verts_b, cube_tris_b, transform = svt.Transforms.Translate([+1.0, 0.0, 0.0]))

# Mesh 4 - line example
mesh4 = scene.create_mesh()
Nsegs = 7000
positions = np.cumsum(np.random.rand(Nsegs, 3) * 0.2, axis = 0)
colored_points = np.concatenate((positions, np.random.rand(Nsegs, 3)), axis = 1)
mesh4.add_lines(colored_points[0:-1, :], colored_points[1:, :])
mesh4.add_camera_frustum(color = svt.Color(1.0,1.0,0.0))

# Let's create two Canvases this time
canvas1 = scene.create_canvas_3d(width = 300, height = 300)
canvas2 = scene.create_canvas_3d(width = 300, height = 300)

# We can link their keyboard/mouse/etc. input events to keep the views in sync
scene.link_canvas_events(canvas1, canvas2)

# And we can specify that certain named "mesh collections" should have user-controlled visibility and opacity
# Meshs without mesh_collection set, or without specified visibilities will always be visible and opaque
canvas1.set_layer_settings({"Coords" : { "opacity" : 0 }, "Sphere+" : { "opacity" : 1 }})

# A Frame contains an array of meshes
frame11 = canvas1.create_frame(meshes = [mesh1, mesh2]) # Note that Frames are created from the Canvas not the Scene
frame21 = canvas2.create_frame(meshes = [mesh2, mesh3])
frame22 = canvas2.create_frame(meshes = [mesh4, mesh1])

# ScenePic has told Jupyter how to display scene objects
scene

In [ ]:
# Tutorial 6 - Images and Textures

# Scene is the top level container in ScenePic
scene = svt.Scene()

# Create and populate an Image object
image1 = scene.create_image(image_id = "PolarBear")
image1.load(asset_path("PolarBear.png")) # This will preserve the image data in compressed PNG format

# Create a texture map
texture = scene.create_image(image_id = "texture")
texture.load(asset_path("uv.png")) # we can use this image to skin meshes

# Example of a mesh that is defined in camera space not world space
# This will not move as the virtual camera is moved with the mouse
cam_space_mesh = scene.create_mesh(shared_color = svt.Color(1.0, 0.0, 0.0), camera_space = True)
cam_space_mesh.add_sphere(transform = np.dot(svt.Transforms.Translate([10, -10, -20.0]), svt.Transforms.Scale(1.0)))

# Some textured primitives
sphere = scene.create_mesh(texture_id=texture.image_id, nn_texture = False)
sphere.add_icosphere(steps=4)

cube = scene.create_mesh(texture_id=texture.image_id)
transform = svt.Transforms.translate([-1, 0, 0]) @ svt.Transforms.scale(0.5)
cube.add_cube(transform=transform)

# Show images in 3D canvas
canvas = scene.create_canvas_3d(shading=svt.Shading(bg_color=svt.Colors.White))
mesh1 = scene.create_mesh(texture_id = "PolarBear")
#mesh1.add_cube(color=svt.Colors.Blue)#
mesh1.add_image() # Adds image in canonical position

# Add an animation that rigidly transforms each image
n_frames = 20
for i in range(n_frames):
    angle = 2 * math.pi * i / n_frames
    c, s = math.cos(angle), math.sin(angle)
    
    # Create a focus point that allows you to "lock" the camera's translation and optionally orientation by pressing the "l" key
    axis = np.array([1.0, 0.0, 1.0])
    axis /= np.linalg.norm(axis)
    focus_point = svt.FocusPoint([c,s,0], orientation_axis_angle = axis * angle)
    
    mesh = scene.create_mesh()
    mesh.add_coordinate_axes(transform = np.dot(svt.Transforms.Translate(focus_point.position), svt.Transforms.RotationMatrixFromAxisAngle(axis, angle)))
    
    im_size = 15
    im_data = np.random.rand(im_size, im_size, 4)
    im_data[:,:,3] = 0.5 + 0.5 * im_data[:,:,3]
    
    imageB = scene.create_image()
    imageB.from_numpy(im_data) # Converts data to PNG format
    meshB = scene.create_mesh(texture_id = imageB, is_billboard = True, use_texture_alpha=True)
    meshB.add_image(transform = np.dot(svt.Transforms.Scale(2.0), svt.Transforms.Translate([0,0,-1])))
   
    frame = canvas.create_frame(focus_point = focus_point)
    frame.add_mesh(mesh1, transform = svt.Transforms.Translate([c,s,0]))
    frame.add_mesh(meshB, transform = np.dot(svt.Transforms.Scale(i * 1.0 / n_frames), svt.Transforms.Translate([-c,-s,0])))
    frame.add_mesh(cam_space_mesh)
    frame.add_mesh(sphere, transform=svt.Transforms.rotation_about_y(np.pi * 2 * i / n_frames))
    frame.add_mesh(cube, transform=svt.Transforms.rotation_about_y(-np.pi * 2 * i / n_frames))
    frame.add_mesh(mesh)
    
# Save result as self-contained HTML file
scene.save_as_html("Redistributable.html", embed_library = False)

# Show Scene
#print scene.get_script()
scene

In [ ]:
# Tutorial 7 - 2D canvases
# Scene is the top level container in ScenePic
scene = svt.Scene()

# Load an image
image1 = scene.create_image(image_id = "PolarBear")
image1.load(asset_path("PolarBear.png")) # This will preserve the image data in compressed PNG format

# Create and populate an Image object
image2 = scene.create_image(image_id = "Random")
image2.from_numpy(np.random.rand(20, 30, 3) * 128 / 255.0) # Converts data to PNG format

# Create a 2D canvas demonstrating different image positioning options
canvas1 = scene.create_canvas_2d(width = 400, height = 300, background_color = svt.Colors.White)
canvas1.create_frame().add_image(image1, "fit")
canvas1.create_frame().add_image(image1, "fill")
canvas1.create_frame().add_image(image1, "stretch")
canvas1.create_frame().add_image(image1, "manual", x = 50, y= 50, scale = 0.3)

# You can composite images and primitives too
canvas2 = scene.create_canvas_2d(width = 300, height = 300)
f = canvas2.create_frame()
f.add_image(image2, "fit")
f.add_image(image1, "manual", x = 30, y= 30, scale = 0.2)
f.add_circle(200, 200, 40, fill_color = svt.Colors.Black, line_width = 10, line_color = svt.Colors.Blue)
f.add_rectangle(200, 100, 50, 25, fill_color = svt.Colors.Green, line_width = 0)
f.add_text("Hello World", 30, 100, svt.Colors.White, 100, "segoe ui light")

scene.framerate = 2

scene

In [ ]:
# Tutorial 8 - a mix of transparent and opaque objects, with labels
from random import *
seed(55)

scene = svt.Scene()
canvas = scene.create_canvas_3d(width = 700, height = 700)
frame = canvas.create_frame()

# Create a mesh that we'll turn in to a point-cloud using enable_instancing()
layer_settings = { "Labels" : { "opacity" : 1.0 }}
N = 20
for i in range(N):
    # Sample object
    geotype = randint(0, 1)
    color = svt.Color(random(), random(), random())
    size = 0.3 * random() + 0.2
    position = 3.0 * np.array([random(), random(), random()]) - 1.5
    opacity = 1.0 if randint(0,1) == 0 else uniform(0.45, 0.55)
    
    # Generate geometry
    layer_id = "Layer" + str(i)
    mesh = scene.create_mesh(shared_color = color, layer_id = layer_id)
    layer_settings[layer_id] = { "opacity" : opacity }
    if geotype == 0:
        mesh.add_cube()
    elif geotype == 1:
        mesh.add_sphere()
    mesh.apply_transform(svt.Transforms.Scale(size)) # Scale the primitive
    mesh.apply_transform(svt.Transforms.Translate(position))
    frame.add_mesh(mesh)
    
    # Add label
    text = "{0:0.2f} {1:0.2f} {2:0.2f} {3:0.2f}".format(color[0], color[1], color[2], opacity)
    horizontal_align = ["left", "center", "right"][randint(0,2)]
    vertical_align = ["top", "middle", "bottom"][randint(0,2)]
    if geotype == 0:
        if horizontal_align != "center" and vertical_align != "middle":
            offset_distance = size * 0.7
        else:
            offset_distance = size * 0.9
    else:
        if horizontal_align != "center" and vertical_align != "middle":
            offset_distance = size * 0.5 * 0.8
        else:
            offset_distance = size * 0.6
    label = scene.create_label(text = text, color = color, layer_id = "Labels", font_family = "consolas", size_in_pixels = 80 * size, offset_distance = offset_distance, vertical_align = vertical_align, horizontal_align = horizontal_align)
    frame.add_label(label = label, position = position)
   

canvas.set_layer_settings(layer_settings)
    
#scene.save_as_html("Temp.html", embed_library=True)
scene

In [ ]:
# Tutorial 9 - mesh animation

# let's create our mesh to get started
scene = svt.Scene()
canvas = scene.create_canvas_3d(width=700, height=700)

# Load a mesh to animate
jelly_mesh = svt.load_obj(asset_path("jelly.obj"))
texture = scene.create_image("texture")
texture.load(asset_path("jelly.png"))

# create a base mesh for the animation. The animation
# will only change the vertex positions, so this mesh
# is used to set everything else, e.g. textures.
base_mesh = scene.create_mesh("jelly_base")
base_mesh.texture_id = texture.image_id
base_mesh.use_texture_alpha = True
base_mesh.add_mesh(jelly_mesh)

# this mesh will only change in a rigid fashion (i.e. one instance)
marble = scene.create_mesh("plate", shared_color=svt.Colors.White)
marble.add_sphere(transform=svt.Transforms.Scale(0.4))

for i in range(60):
    # animate the wave mesh by updating the vertex positions
    positions = jelly_mesh.positions.copy()
    delta_x = (positions[:, 0] + 0.0838 * i) * 10
    delta_z = (positions[:, 2] + 0.0419 * i) * 10
    positions[:, 1] = positions[:, 1] + 0.1 * (np.cos(delta_x) + np.sin(delta_z))
    
    # we create a mesh update with the new posiitons. We can use this mesh update
    # just like a new mesh, because it essentially is one, as ScenePic will create
    # a new mesh from the old one using these new positions.
    mesh_update = scene.update_mesh("jelly_base", positions)
    frame = canvas.create_frame(meshes=[mesh_update])

    # this is a more simple form of animation using rigid transforms
    marble_y = np.sin(0.105 * i)
    frame.add_mesh(marble, transform=svt.Transforms.Translate([0, marble_y, 0]))

scene.quantize_updates()
scene

In [ ]:
# Tutorial 10 - camera movement

# in this tutorial we will show how to create per-frame camera movement.
# while the user can always choose to override this behavior, having a
# camera track specified can be helpful for demonstrating particular
# items in 3D. We will also show off the flexible GLCamera class.

scene = svt.Scene()
spin_canvas = scene.create_canvas_3d("spin")
spiral_canvas = scene.create_canvas_3d("spiral")

# let's create some items in the scene so we have a frame of reference
polar_bear = scene.create_image(image_id="polar_bear")
polar_bear.load(asset_path("PolarBear.png"))
uv_texture = scene.create_image(image_id = "texture")
uv_texture.load(asset_path("uv.png"))

cube = scene.create_mesh("cube", texture_id=polar_bear.image_id)
cube.add_cube()
sphere = scene.create_mesh("sphere", texture_id=uv_texture.image_id)
sphere.add_icosphere(steps=4, transform=svt.Transforms.translate([0, 1, 0]))

# for visualization we'll also create some camera frustums
spin_frustum = scene.create_mesh("spin_frustum", shared_color=svt.Colors.Red)
spiral_frustum = scene.create_mesh("spiral_frustum", shared_color=svt.Colors.Green)
# Adjust from GL to Standard
spin_frustum.add_camera_frustum(depth=-1, transform=svt.Transforms.rotation_about_z(np.pi), aspect_ratio=1)
spiral_frustum.add_camera_frustum(depth=-1, transform=svt.Transforms.rotation_about_z(np.pi), aspect_ratio=1)

num_frames = 60
for i in range(num_frames):
    spin_frame = spin_canvas.create_frame()
    spin_frame.add_meshes([cube, sphere])
    
    angle = i*np.pi*2/num_frames

    # for the first camera we will spin in place on the Z axis
    rotation = svt.Transforms.rotation_about_z(angle)
    spin_frame.camera = svt.Camera(center=[0, 0, 4], rotation=rotation, fov_y_degrees=30.0)
    
    spiral_frame = spiral_canvas.create_frame()
    spiral_frame.add_meshes([cube, sphere])

    # for the second camera, we will spin the camera in a spiral around the scene
    # we can do this using the look-at initialization, which provides a straightforward
    # "look at" interface for camera placement.
    camera_center = [4*np.cos(angle), i*4/num_frames - 2, 4*np.sin(angle)]
    spiral_frame.camera = svt.Camera(camera_center, look_at=[0, 0.5, 0])

    # add the camera frustums
    spiral_frame.add_mesh(spiral_frustum, transform=spiral_frame.camera.camera_to_world)
    spiral_frame.add_mesh(spin_frustum, transform=spin_frame.camera.camera_to_world)
    spin_frame.add_mesh(spiral_frustum, transform=spiral_frame.camera.camera_to_world)
    spin_frame.add_mesh(spin_frustum, transform=spin_frame.camera.camera_to_world)
    

scene.link_canvas_events(spin_canvas, spiral_canvas)
scene

In [ ]:
# Tutorial 11 - audio tracks

# in this tutorial we'll show how to attach audio tracks to canvases. ScenePic
# supports any audio file format supported by the browser.

def _set_audio(scene, canvas, path):
    audio = scene.create_audio()
    audio.load(path)
    canvas.media_id = audio.audio_id

scene = svt.Scene()

names = ["red", "green", "blue"]
colors = [svt.Colors.Red, svt.Colors.Green, svt.Colors.Blue]
frequencies = [0, 1, 0.5]

graph = scene.create_graph("graph", width=900, height=150)
for name, color, frequency in zip(names, colors, frequencies):
    mesh = scene.create_mesh()
    mesh.add_cube(color)
    canvas = scene.create_canvas_3d(name, width=300, height=300)
    _set_audio(scene, canvas, asset_path(name + ".ogg"))
    values = []

    for j in range(60):
        frame = canvas.create_frame()
        scale = math.sin(j * 2 * math.pi * frequency / 30)
        frame.add_mesh(mesh, svt.Transforms.scale((scale + 1) / 2 + 0.5))
        values.append(scale)

    graph.add_sparkline(name, values, color)
    graph.media_id = canvas.media_id

names.append("graph")
scene.grid("600px", "1fr auto", "1fr 1fr 1fr")
scene.place("graph", "2", "1 / span 3")
scene.link_canvas_events(*names)
scene

In [ ]:
# Tutorial 11 - video

# It is also possible to attach videos to ScenePic scenes. Once attached, you can draw the
# frames of those videos to canvases in the same way as images, and can draw the same
# video to multiple frames. Once a media file (video or audio) has been attached to a
# canvas, that file will be used to drive playback. In practical terms, this means that
# ScenePic will display frames such that they line up with the timestamps of the video
# working on the assumption that ScenePic frames are displayed at 30 FPS.


def _angle_to_pos(angle, radius):
    return np.cos(angle) * radius + 200, np.sin(angle) * radius + 200


scene = svt.Scene()

video = scene.create_video()
video.load(asset_path("circles.mp4"))

tracking = scene.create_canvas_2d("tracking", background_color=svt.Colors.White)
tracking.media_id = video.video_id

multi = scene.create_canvas_2d("multi", background_color=svt.Colors.White)
multi.media_id = video.video_id

angles = np.linspace(0, 2 * np.pi, 360, endpoint=False)
for angle in angles:
    frame = tracking.create_frame()
    frame.add_video(layer_id="video")

    red_pos = _angle_to_pos(angle, 160)
    frame.add_rectangle(red_pos[0] - 11, red_pos[1] - 11, 22, 22, [255, 0, 0], 2, layer_id="rect")
    frame.add_circle(red_pos[0], red_pos[1], 10, fill_color=[255, 0, 0], layer_id="dot")

    green_pos = _angle_to_pos(-2*angle, 80)
    frame.add_rectangle(green_pos[0] - 11, green_pos[1] - 11, 22, 22, [0, 255, 0], 2, layer_id="rect")
    frame.add_circle(green_pos[0], green_pos[1], 10, fill_color=[0, 255, 0], layer_id="dot")

    blue_pos = _angle_to_pos(4*angle, 40)
    frame.add_rectangle(blue_pos[0] - 11, blue_pos[1] - 11, 22, 22, [0, 0, 255], 2, layer_id="rect")
    frame.add_circle(blue_pos[0], blue_pos[1], 10, fill_color=[0, 0, 255], layer_id="dot")

    frame = multi.create_frame()
    frame.add_video("manual", red_pos[0] - 40, red_pos[1] - 40, 0.2, layer_id="red")
    frame.add_video("manual", green_pos[0] - 25, green_pos[1] - 25, 0.125, layer_id="green")
    frame.add_video("manual", 160, 160, 0.2, layer_id="blue")

tracking.set_layer_settings({
    "rect": {"render_order": 0},
    "video": {"render_order": 1},
    "dot": {"render_order": 2}
})

scene.link_canvas_events("tracking", "multi")
scene

In [ ]:
# Tutorial 12 - Multiview Visualization

# One common and useful scenario for ScenePic is to visualize the result of multiview 3D reconstruction.
# In this tutorial we'll show how to load some geometry, assocaited camera calibration
# information, and images to create a visualization depicting the results.

def _load_camera(camera_info):
    # this function loads an "OpenCV"-style camera representation
    # and converts it to a GL style for use in ScenePic
    location = np.array(camera_info["location"], np.float32)
    euler_angles = np.array(camera_info["rotation"], np.float32)
    rotation = svt.Transforms.euler_angles_to_matrix(euler_angles, "XYZ")
    translation = svt.Transforms.translate(location)
    extrinsics = translation @ rotation
    world_to_camera = svt.Transforms.gl_world_to_camera(extrinsics)
    aspect_ratio = camera_info["width"] / camera_info["height"]
    projection = svt.Transforms.gl_projection(camera_info["fov"], aspect_ratio, 0.01, 100)

    return svt.Camera(world_to_camera, projection)


def _load_cameras():
    with open(asset_path("cameras.json")) as file:
        cameras = json.load(file)
        return [_load_camera(cameras[key])
                for key in cameras]


scene = svt.Scene()

# load the fitted cameras
cameras = _load_cameras()

# this textured cube will stand in for a reconstructed mesh
texture = scene.create_image("texture")
texture.load(asset_path("PolarBear.png"))
cube = scene.create_mesh("cube")
cube.texture_id = texture.image_id
cube.add_cube(transform=svt.Transforms.scale(2))

# construct all of the frustums
# and camera images
frustums = scene.create_mesh("frustums", layer_id="frustums")
colors = [svt.Colors.Red, svt.Colors.Green, svt.Colors.Blue]
paths = [asset_path(name) for name in ["render0.png", "render1.png", "render2.png"]]
camera_images = []
images = []

for i, (color, path, camera) in enumerate(zip(colors, paths, cameras)):
    image = scene.create_image(path)
    image.load(path)
    frustums.add_camera_frustum(camera, color)
    
    image_mesh = scene.create_mesh("image{}".format(i),
                                   layer_id="images",
                                   shared_color=svt.Colors.Gray,
                                   double_sided=True,
                                   texture_id=image.image_id)
    image_mesh.add_camera_image(camera)
    
    images.append(image)
    camera_images.append(image_mesh)

# create one canvas for each camera to show the scene from
# that camera's viewpoint
width = 640
for i, camera in enumerate(cameras):
    height = width / camera.aspect_ratio
    canvas = scene.create_canvas_3d("hand{}".format(i), width, height, camera=camera)
    frame = canvas.create_frame()
    frame.add_mesh(cube)
    frame.add_mesh(frustums)
    frame.camera = camera
    for cam_mesh in camera_images:
        frame.add_mesh(cam_mesh)

scene
